In [1]:
import pickle
import pandas as pd
import os
import torch
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime
from datetime import timedelta
from dateutil import tz
import math
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.linear_model import LogisticRegression
import fasttrack as ft
import importlib
import featurecreations
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder
from ft_sec_key import SECKEY
import logging

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
betfair_df = pickle.load(open(r"./DATA/df-betfairSP.npy", 'rb'))

ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'